# DAT257x: Reinforcement Learning Explained

## Lab 4: Dynamic Programming

### Exercise 4.1 Policy Evaluation with 2 Arrays

Policy Evaluation calculates the value function for a policy, given the policy and the full definition of the associated Markov Decision Process.  The full definition of an MDP is the set of states, the set of available actions for each state, the set of rewards, the discount factor, and the state/reward transition function.

In [1]:
import test_dp               # required for testing and grading your code
import gridworld_mdp as gw   # defines the MDP for a 4x4 gridworld

The gridworld MDP defines the probability of state transitions for our 4x4 gridworld using a "get_transitions()" function.  

Let's try it out now, with state=2 and all defined actions.

In [2]:
# try out the gw.get_transitions(state, action) function

state = 2
actions = gw.get_available_actions(state)

for action in actions:
    transitions = gw.get_transitions(state=state, action=action)

    # examine each return transition (only 1 per call for this MDP)
    for (trans) in transitions:
        next_state, reward, probability = trans    # unpack tuple
        print("transition("+ str(state) + ", " + action + "):", "next_state=", next_state, ", reward=", reward, ", probability=", probability)

transition(2, up): next_state= 2 , reward= -1 , probability= 1
transition(2, down): next_state= 6 , reward= -1 , probability= 1
transition(2, left): next_state= 1 , reward= -1 , probability= 1
transition(2, right): next_state= 3 , reward= -1 , probability= 1


**Implement the algorithm for Iterative Policy Evaluation using the 2 array approach**.  In the 2 array approach, one array holds the value estimates for each state computed on the previous iteration, and one array holds the value estimates for the states computing in the current iteration.

A empty function **policy_eval_two_arrays** is provided below; implement the body of the function to correctly calculate the value of the policy using the 2 array approach.  The function defines 5 parameters - a definition of each parameter is given in the comment block for the function.  For sample parameter values, see the calling code in the cell following the function.

In [3]:
import inspect
print(inspect.signature(gw.get_available_actions))
print(inspect.signature(gw.get_transitions))

(state)
(state, action)


In [20]:
x1 = [1, 2, 3, 4, 5]
x2 = [6, 7, 8, 9, 10]

print(x2)
x1[3] = 25
x1[4] = 35
x2 = x1
print(x2)

[6, 7, 8, 9, 10]
[1, 2, 3, 25, 35]


In [23]:
import numpy as np
from numpy import linalg as LA

def policy_eval_two_arrays(state_count, gamma, theta, get_policy, get_transitions):
    """
    This function uses the two-array approach to evaluate the specified policy for the specified MDP:
    
    'state_count' is the total number of states in the MDP. States are represented as 0-relative numbers.
    
    'gamma' is the MDP discount factor for rewards.
    
    'theta' is the small number threshold to signal convergence of the value function (see Iterative Policy Evaluation algorithm).
    
    'get_policy' is the stochastic policy function - it takes a state parameter and returns list of tuples, 
        where each tuple is of the form: (action, probability).  It represents the policy being evaluated.
        
    'get_transitions' is the state/reward transiton function.  It accepts two parameters, state and action, and returns
        a list of tuples, where each tuple is of the form: (next_state, reward, probabiliity).  
        
    """
    
    Vk = state_count*[0]
    Vkplus1 = state_count*[0]
    
    #
    # INSERT CODE HERE to evaluate the policy using the 2 array approach 
    #
    
    while True:
        
        delta = 0
        
        for i in range(state_count):
        
            # Get policy for each state and unpack

            action_probs = get_policy(i)
            
            # Loop over each tuple in action_probs array

            out_sum = 0
            
            
            for (action_prob) in action_probs:
                
                # Unpack each action_prob tuple and get action and its probability
                
                action, prob = action_prob

                # Use action, to get array of transitions

                transitions = gw.get_transitions(state=i, action=action)

                # loop over eacch possible transition to get next state, reward, and probability of transition

                inner_sum = 0
                
                for (trans) in transitions:

                    next_state, reward, probability = trans    # unpack tuple
                    
                    # Calculate sigma(r, sprime) * p(r, sprime | s, a)
                    
                    inner_sum = inner_sum + probability * (reward + gamma * Vk[next_state])
                    
                # calculate sigma(pi(a | s) * sigma(r, sprime) * p(r, sprime | s, a))
                    
                out_sum = out_sum + prob * inner_sum
                
            Vkplus1[i] = out_sum
            
            delta = max(delta, abs(Vk[i] - Vkplus1[i]))
            
        # check exit condition
        
 #       if LA.norm((np.array(Vk) - np.array(Vkplus1)), 1) < theta:
        if delta < theta:
            
            # Be careful. This is dangerous because Vkplus1 will and Vk no reference the same list whose elements will be modified 
            # next iteration. Instead make a copy of Vkplus1 and then assign to Vk so it remains unchanged until this assignment
#            Vk = Vkplus1 
            Vk = Vkplus1.copy()
            break
            
        else:
            
            Vk = Vkplus1.copy()
                
    return Vk

First, test our function using the MDP defined by gw.* functions.

In [24]:
def get_equal_policy(state):
    # build a simple policy where all 4 actions have the same probability, ignoring the specified state
    policy = ( ("up", .25), ("right", .25), ("down", .25), ("left", .25))
    return policy

n_states = gw.get_state_count()

# test our function
values = policy_eval_two_arrays(state_count=n_states, gamma=.9, theta=.001, get_policy=get_equal_policy, \
    get_transitions=gw.get_transitions)

print("Values=", values)

Values= [0.0, -5.274709263277986, -7.123800104889248, -7.64536148969558, -5.274709263277987, -6.602238720082915, -7.17604178238719, -7.1238001048892485, -7.1238001048892485, -7.176041782387191, -6.602238720082915, -5.274709263277986, -7.645361489695581, -7.1238001048892485, -5.274709263277986]


**Expected output from running above cell:**

`
Values= [0.0, -5.274709263277986, -7.123800104889248, -7.64536148969558, -5.274709263277987, -6.602238720082915, -7.17604178238719, -7.1238001048892485, -7.1238001048892485, -7.176041782387191, -6.602238720082915, -5.274709263277986, -7.645361489695581, -7.1238001048892485, -5.274709263277986]
`

In [6]:
import numpy as np
a = np.append(values, 0)
np.reshape(a, (4,4))

array([[ 0.        , -1.675     , -2.051875  , -2.13667187],
       [-1.675     , -2.20375   , -2.40751563, -2.47244219],
       [-2.051875  , -2.40751562, -2.53338203, -2.35131045],
       [-2.13667188, -2.47244219, -2.35131045,  0.        ]])

Now, test our function using the test_dp helper.  The helper also uses the gw MDP, but with a different gamma value.
If our function passes all tests, a passcode will be printed.

In [ ]:
# test our function using the test_db helper
test_dp.policy_eval_two_arrays_test( policy_eval_two_arrays ) 